## Testing the agent

In [33]:
import os
from dotenv import load_dotenv

from tools.get_domain_info import get_domain_info
from tools.check_verified_reviews import extract_with_diffbot
from tools.check_community_discussion import check_reddit_reviews
from tools.final_report import submit_final_report, TrustReport

from typing import List, Literal
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
prompt_file_path = 'system_prompt.md'

In [35]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [27]:
all_tools = [
   get_domain_info,
   extract_with_diffbot,
   check_reddit_reviews,
   submit_final_report
]

In [28]:
def get_system_prompt(file_path):
    try:
        with open(file_path, 'r', encoding="UTF-8") as file:
            system_prompt = file.read()
            
        # Access the prompt text
        # system_prompt = config['agent_config']['system_prompt'].strip()
        print("Agent prompt loaded successfully!")
        return system_prompt
      
    except FileNotFoundError:
        print(f"Error: System prompt file not found at {prompt_file_path}.")

    except KeyError as e:
        print(f"Error: Missing key in system prompt file: {e}. Check the 'agent_config' and 'system_prompt' keys.")


SYSTEM_PROMPT = get_system_prompt(prompt_file_path)

Agent prompt loaded successfully!


In [29]:
llm = ChatOpenAI(
   model="gpt-5-nano"
   )

In [31]:
# messages = [
#     (
#         "system",
#         "You are a helpful assistant that translates English to French. Translate the user sentence.",
#     ),
#     ("human", "I love programming."),
# ]
# ai_msg = llm.invoke(messages)
# ai_msg

In [34]:
# THE PYDANTIC "FINAL ANSWER" SCHEMA ---

class TrustReport(BaseModel):
   """The structured trust report for the web extension."""
   risk_level: Literal["Low", "Medium", "High", "Critical"] = Field(..., description="The overall risk level assessed.")
   explanation: str = Field(..., description="A 1-2 sentence summary explaining the risk level for the user.")
   red_flags: List[str] = Field(..., description="A bullet-point list of 3-5 specific red flags found.")
   confidence: Literal["High", "Medium", "Low"] = Field(..., description="The confidence level of this assessment (e.g., 'Low' if data is scarce).")

In [36]:
agent = create_agent(model= llm, 
                     tools = all_tools,
                     system_prompt= SYSTEM_PROMPT,
                     response_format= TrustReport
                    )

In [37]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Is this domain legit enroutejewelry.com"}]})

In [39]:
response



{'messages': [HumanMessage(content='Is this domain legit enroutejewelry.com', additional_kwargs={}, response_metadata={}, id='beaf0a95-1d84-467b-8429-c4fb4fdb1e65'),
  AIMessage(content='', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 479, 'prompt_tokens': 1823, 'total_tokens': 2302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcI2McEduV13JMTj0yk8QT9wKnGd4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--213ae66e-5f18-4401-b176-89e058df5e2e-0', tool_calls=[{'name': 'get_domain_info', 'args': {'url': 'https://enroutejewelry.com'}, 'id': 'call_TwP9hExHwp1IlbAJhd5GzhFk', 'type': 'tool_call'}], usage_metadata={'inpu

In [40]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Is this domain legit modlily.com"}]})
response

{'messages': [HumanMessage(content='Is this domain legit modlily.com', additional_kwargs={}, response_metadata={}, id='dc7e4c7e-5c19-43d8-8de1-df401f01189f'),
  AIMessage(content='', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 797, 'prompt_tokens': 1821, 'total_tokens': 2618, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CcI6QwpvO8f23TjCPbbegXuPoHqPi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ac4327f9-83e8-415b-9937-5fbb97588cdc-0', tool_calls=[{'name': 'get_domain_info', 'args': {'url': 'https://modlily.com'}, 'id': 'call_MwGKhIQQapKAmeik8MwVajX2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 